## Chapter 4　言語処理特有の深層学習の発展

---
### 4.1　注意機構
複数のベクトルがあるとき、どのベクトルを重要視するかも含めて学習させる仕組みを  
注意機構（attention mechanism）、あるいは注意と呼ぶ。

このうち、複数ベクトルの重み付き平均を使う方法をソフト注意機構（soft attention mechanism）と呼ぶ。

---
#### ソフト注意機構
長さ $I$ の入力系列 $\{\mathbf{x}_{1},\ldots,\mathbf{x}_{I}\}$ に対し、各時刻で符号化されたベクトルを $\{ \mathbf{h}_{1},\ldots,\mathbf{h}_{I}\}$ とおく。  
この計算は、RNN の遷移関数を $\Psi^{(s)}$ とするとき次の式による。

\begin{align*}
\mathbf{h}_{i}^{(s)} = \Psi^{(s)} ( \mathbf{x}_{i}, \mathbf{h}_{i-1}^{(s)})
\end{align*}

通常の系列変換モデルでは最後の状態出力 $\mathbf{h}_{I}$ のみを用いるが、
ここではより直接的に情報を伝播する方法を考える。

復号化器でも RNN により隠れ状態ベクトル $\mathbf{h}_{j}^{(t)}$ を計算する。

\begin{align*}
\mathbf{h}_{j}^{(t)} = \Psi^{(t)}(\mathbf{y}_{j}, \mathbf{h}_{j-1}^{(t)})
\end{align*}

いま、復号化器が $j$ 番目の単語を推定するときに、  
復号化器の $i$ 番目の隠れ状態ベクトル $\mathbf{h}_{i}^{(s)}$ の重要度を示すスカラー値の重みを $a_{i} \in \mathbb{R}$ とする。  
$a_{i}$ による $\mathbf{h}_{i}^{(s)}$ の重み付き平均 $\bar{\mathbf{h}}$ を

\begin{align*}
\bar{\mathbf{h}} = \sum_{\tilde{i}=1}^{I} a_{\tilde{i}}\mathbf{h}_{\tilde{i}^{(s)}}
\end{align*}

とする。

復号化器が $j$ 番目の単語の予測に $\bar{\mathbf{h}}$ を利用するため、$\mathbf{h}_{j}^{(t)}$ の代わりに次式で定義される $\hat{\mathbf{h}}_{j}^{(t)}$ を利用する。

\begin{align*}
\hat{\mathbf{h}}_{j}^{(t)} = \operatorname{tanh} \left( \mathbf{W}^{(a)} \left[ \begin{array} \,\bar{\mathbf{h}}\\ \mathbf{h}_{j}^{(t)} \end{array} \right] \right)
\end{align*}

$\{a_{1},\ldots,a_{I}\}$ もモデル内で計算を行う。  
まず関数 $\Omega$ によって $\mathbf{h}_{i}^{(s)}$ と $\mathbf{h}_{j}^{(t)}$ の間の重み $e_{i}$ を計算する。

\begin{align*}
e_{i} = \Omega(\mathbf{h}_{i}^{(s)}, \mathbf{h}_{j}^{(t)})
\end{align*}

次に、$a_{i}$ は総和が１になるようにソフトマックス関数で正規化する。

\begin{align*}
a_{i} = \frac{\exp(e_{i})}{\sum_{\tilde{i}=1}^{I} \exp (e_{\tilde{i}})}
\end{align*}

上で使用した関数 $\Omega$ については様々な関数が利用されており、以下は一例。

\begin{align*}
\Omega (\mathbf{h}_{i}^{(s)}, \mathbf{h}_{j}^{(t)}) = \begin{cases}
\mathbf{h}_{i} \cdot \mathbf{h}_{j}^{(t)} \\
\mathbf{h}_{i}^{(s)} \cdot \mathbf{Wh}_{j}^{(t)} \\
\mathbf{v} \cdot \operatorname{tanh} \left( \mathbf{W} \left[ \begin{array}\, \mathbf{h}_{i}^{(s)}\\\mathbf{h}_{j}^{(t)} \end{array} \right] \right)
\end{cases}
\end{align*}

$a_{i}$ は確率として解釈することもでき、  
このとき $\bar{\mathbf{h}}$ は $\{a_{1},\ldots,a_{I}\}$ の分布における $\mathbf{h}_{i}^{(s)}$ の期待値と見なせる。

より一般化する。  
$N$ 個の参照したい情報 $\mathbf{Y} = \{ \mathbf{y}_{1}, \ldots, \mathbf{y}_{N}\}$ （上の例では符号化器の隠れ状態）があり、  
それぞれの重要度、あるいはいずれかが選択される確率 $\{ a_{1},\ldots,a_{N}\}$ を計算する。  
$a_{i}$ の計算については何らかの情報 $\mathbf{c}_{i}$ を用いる（上の例では隠れ状態）。

例えば end-to-end 記憶ネットワークでは記憶を参照するのにソフト注意機構を利用しており

\begin{align*}
a_{i} = \frac{\exp(\Omega(\mathbf{c}_{i}))}{\sum_{i=1}^{I} \exp (\Omega (\mathbf{c}_{\tilde{i}}))}
\end{align*}

を利用して次の通り $\hat{\mathbf{y}}$ を得る。

\begin{align*}
\hat{\mathbf{y}} = \mathbb{E}_{a}[\mathbf{y}] = \sum_{i=1}^{N} a_{i}\mathbf{y}_{i}
\end{align*}



---
#### ハード注意機構
ソフト注意機構では複数の情報源が選択される確率に対しその情報の期待値を確定的に計算したのに対し、  
ハード注意機構では情報源の中から確率的に１つ選択されるものとして計算する。

$N$ 個の参照したい情報 $\mathbf{Y} = \{ \mathbf{y}_{1}, \ldots, \mathbf{y}_{N}\}$ があり、  
確率変数 $X$ が $\{1,\ldots,N\}$ の中から確率的に１つを選択するものとして  
この時の確率を $P(X=i) = a_{i}$ とおく。すなわち

\begin{align*}
P(X=i)=a_{i}
\end{align*}

である。$\mathbf{Y}$ の中から $X$ を使って無作為に１つ選択された結果を

\begin{align*}
\hat{\mathbf{y}} = \mathbf{y}X
\end{align*}

とする。

ハード注意機構を利用した学習で目的関数 $f(\hat{\mathbf{y}})$ が計算されたとして、これを最小化する。  
$\hat{\mathbf{y}}$ および $f(\hat{\mathbf{y}})$ は確率的に振る舞うため、期待値 $\mathbb{E}[f(\hat{\mathbf{y}})]$ を最小化の対象とする。  
期待値の微分は次の通り。

\begin{align*}
\nabla \mathbb{E}[f(\hat{\mathbf{y}})] &= \nabla \sum_{x=1}^{N} f( \mathbf{y}_{x}) a_{x} \\
&= \sum_{x=1}^{N} \nabla f(\mathbf{y}_{x}) a_{x} + \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x} \\
&= \mathbb{E}[\nabla f(\hat{\mathbf{y}})] + \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x}
\end{align*}

いずれの項もすべての $x$ について計算することから計算の負荷が高く、近似が必要となる。  
第１項についてはモンテカルロ法を用いる、つまり有限個の標本で次のように近似する。

\begin{align*}
\mathbb{E}[\nabla f(\hat{\mathbf{y}})] \approx \frac{1}{T}\sum_{i=1}^{T} \nabla f(\mathbf{y}_{\bar{x}_{i}})
\end{align*}

第２項についても変形すると期待値の式に変形できるので、モンテカルロ法が適用可能である。

\begin{align*}
\sum_{x=1}^{N} f (\mathbf{y}_{x}) \nabla a_{x} &= \sum_{x=1}^{N} f(\mathbf{y}_{x}) \nabla a_{x} \frac{a_{x}}{a_{x}} \\
&= \mathbb{E} \left[ f( \hat{\mathbf{y}} ) \frac{\nabla a_{x}}{a_{x}} \right] \\
&= \mathbb{E} [ f( \hat{\mathbf{y}}) \nabla \log a_{x}]
\end{align*}

以上により

\begin{align*}
\nabla \mathbb{E}[f(\hat{\mathbf{y}})] &= \mathbb{E}[\nabla f(\hat{\mathbf{y}}) + f( \hat{\mathbf{y}}) \nabla \log a_{x}] \\
&\approx \frac{1}{T}\sum_{i=1}^{T} \{ \nabla f(\mathbf{y}_{\bar{x}_{i}}) + f(\mathbf{y}_{\bar{x}_{i}}) \nabla \log a_{\bar{x}_{i}} \}
\end{align*}

となる。

なお実用上、標本から計算される $f( \mathbf{y}_{\bar{x}_{i}}) \nabla \log a_{\bar{x}_{i}}$ の分散が大きい時に学習できないことがある。  
そこで分散を抑えることを考える。定数 $b$ を用いて

\begin{align*}
\mathbb{e}[f( \hat{\mathbf{y}} ) \nabla \log a_{x}] &= \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}] + \mathbb{E}[ b \nabla \log a_{x}] \\
&= \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}] + b \mathbb{E}[\nabla \log a_{x}] \\
\mathbb{E}[\nabla \log a_{x}] &= \sum_{x=1}^{N} \nabla a_{x} = \nabla \sum_{x}^{N} a_{x}
\end{align*}

と変形する。$a_{x}$ は確率値であるから、$ \sum_{x=1}^{N}a_{x}=1, \nabla \sum_{x=1}^{N}a_{x}=0$ である。  
よって任意の定数 $b \in \mathbb{R}$ について

\begin{align*}
\mathbb{E}[f( \hat{\mathbf{y}} ) \nabla \log a_{x}] = \mathbb{E}[(f( \hat{\mathbf{y}} ) -b ) \nabla \log a_{x}]
\end{align*}

が成り立ち、この $b$ をベースラインと呼ぶ。  
ベースラインとしては $b=\mathbb{E}[f( \hat{\mathbf{y}})]$ がよく用いられる。

---
その他の注意機構としては、局所注意機構（local attention mechanism）などがある。

---
### 4.2　記憶ネットワーク